# 8.1-1

8.1-1. The Childfair Company has three plants producing child push chairs that are to be shipped to four
distribution centers. Plants 1, 2, and 3 produce 12, 17, and 11 shipments per month, respectively. Each
distribution center needs to receive 10 shipments per month. The distance from each plant to the respective
distributing centers is given as shown below (in miles)

| Plant| Dist 1 | Dist 2 | Dist 3 | Dist 4 |
| --- | --- | --- | --- | --- |
| Plant 1  | 800 | 1300 | 400 | 700 |
| Plant 2  | 1100 | 1400 | 600 | 1000 |
| Plant 3  | 600 | 1200 | 800 | 900 |



The freight cost for each shipment is $100 plus 50 cents per mile. How much should be shipped from each plant to each
of the distribution centers to minimize the total shipping cost? (a) Formulate this problem as a transportation
problem by constructing the appropriate parameter table. (b) Draw the network representation of this problem. (c)
Obtain an optimal solution.

In [1]:
import pandas as pd
from docplex.mp.model import Model
import os

In [2]:
# Read in external data
df_arc = pd.read_csv('data_arc.csv')
df_plant = pd.read_csv('data_plant.csv')
df_distribution = pd.read_csv('data_distribution.csv')

In [3]:
# Check
df_arc.head()

,plant,distribution,distance
0,1,1,800
1,1,2,1300
2,1,3,400
3,1,4,700
4,2,1,1100


In [4]:
# Check 
df_plant.head()

,plant,capacity,mileage_cost,shipment_cost
0,1,12,0.5,100
1,2,17,0.5,100
2,3,11,0.5,100


In [5]:
# Check 
df_distribution.head()

,distribution,demand
0,1,10
1,2,10
2,3,10
3,4,10


In [6]:
# Indices/sets
arcs = list((t.plant, t.distribution) for t in df_arc.itertuples())
plants = df_plant['plant']
distributions = df_distribution['distribution']

In [7]:
# Check
arcs

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4)]

In [8]:
# Check
plants

0    1
1    2
2    3
Name: plant, dtype: int64

In [9]:
# Check
distributions

0    1
1    2
2    3
3    4
Name: distribution, dtype: int64

In [10]:
# Parameters
distance = dict([((t.plant, t.distribution), t.distance) for t in df_arc.itertuples()])
capacity = df_plant['capacity']
demand = df_distribution['demand']

In [11]:
# Check
distance

{(1, 1): 800,
 (1, 2): 1300,
 (1, 3): 400,
 (1, 4): 700,
 (2, 1): 1100,
 (2, 2): 1400,
 (2, 3): 600,
 (2, 4): 1000,
 (3, 1): 600,
 (3, 2): 1200,
 (3, 3): 800,
 (3, 4): 900}

In [12]:
# Check 
capacity

0    12
1    17
2    11
Name: capacity, dtype: int64

In [13]:
# Check 
demand

0    10
1    10
2    10
3    10
Name: demand, dtype: int64

In [14]:
# Fixed parameters
shipment_cost = 100
per_mile_cost = .50
max_demand = 10

In [15]:
# Create model
m = Model('8.1-1')
m

In [16]:
# Create decision variables
x = m.integer_var_dict(arcs, name='x')
x

{(1, 1): docplex.mp.Var(type=I,name='x_1_1'),
 (1, 2): docplex.mp.Var(type=I,name='x_1_2'),
 (1, 3): docplex.mp.Var(type=I,name='x_1_3'),
 (1, 4): docplex.mp.Var(type=I,name='x_1_4'),
 (2, 1): docplex.mp.Var(type=I,name='x_2_1'),
 (2, 2): docplex.mp.Var(type=I,name='x_2_2'),
 (2, 3): docplex.mp.Var(type=I,name='x_2_3'),
 (2, 4): docplex.mp.Var(type=I,name='x_2_4'),
 (3, 1): docplex.mp.Var(type=I,name='x_3_1'),
 (3, 2): docplex.mp.Var(type=I,name='x_3_2'),
 (3, 3): docplex.mp.Var(type=I,name='x_3_3'),
 (3, 4): docplex.mp.Var(type=I,name='x_3_4')}

In [23]:
# Objective function
m.minimize(m.sum(distance[ij] * per_mile_cost * x[ij] for ij in arcs) + m.sum(shipment_cost * x[ij] for ij in arcs))

In [24]:
# Check
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 8.1-1

Minimize
 obj: 500 x_1_1 + 750 x_1_2 + 300 x_1_3 + 450 x_1_4 + 650 x_2_1 + 800 x_2_2
      + 400 x_2_3 + 600 x_2_4 + 400 x_3_1 + 700 x_3_2 + 500 x_3_3 + 550 x_3_4
Subject To
 capacity_constraint_1: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_constraint_2: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_constraint_3: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11

Bounds

Generals
 x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 x_3_3 x_3_4
End



In [25]:
# capacity constraint
for i in plants:
    m.add_constraint(m.sum(x[(i, j)] for j in distributions) <= capacity[i - 1], ctname='capacity_constraint_%d' % i)

In [26]:
# Check
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 8.1-1

Minimize
 obj: 500 x_1_1 + 750 x_1_2 + 300 x_1_3 + 450 x_1_4 + 650 x_2_1 + 800 x_2_2
      + 400 x_2_3 + 600 x_2_4 + 400 x_3_1 + 700 x_3_2 + 500 x_3_3 + 550 x_3_4
Subject To
 capacity_constraint_1: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_constraint_2: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_constraint_3: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11
 capacity_constraint_1#3: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_constraint_2#4: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_constraint_3#5: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11

Bounds

Generals
 x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 x_3_3 x_3_4
End



In [27]:
# demand constraint
for j in distributions:
    m.add_constraint(m.sum(x[i, j] for i in plants) >= demand[i - 1], ctname='demand_constraint_%d' % j)


In [28]:
# Check
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 8.1-1

Minimize
 obj: 500 x_1_1 + 750 x_1_2 + 300 x_1_3 + 450 x_1_4 + 650 x_2_1 + 800 x_2_2
      + 400 x_2_3 + 600 x_2_4 + 400 x_3_1 + 700 x_3_2 + 500 x_3_3 + 550 x_3_4
Subject To
 capacity_constraint_1: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_constraint_2: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_constraint_3: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11
 capacity_constraint_1#3: x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 12
 capacity_constraint_2#4: x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 17
 capacity_constraint_3#5: x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 11
 demand_constraint_1: x_1_1 + x_2_1 + x_3_1 >= 10
 demand_constraint_2: x_1_2 + x_2_2 + x_3_2 >= 10
 demand_constraint_3: x_1_3 + x_2_3 + x_3_3 >= 10
 demand_constraint_4: x_1_4 + x_2_4 + x_3_4 >= 10

Bounds

Generals
 x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 x_3_3 x_3_4
End



In [29]:
# Set parameters for solving model
m.parameters.timelimit = 120
m.parameters.mip.strategy.branch = 1
m.parameters.mip.tolerances.mipgap = 0.15

In [30]:
# Solve model
soln = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Found incumbent of value 22500.000000 after 0.01 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 0 columns.
Reduced MIP has 7 rows, 12 columns, and 24 nonzeros.
Reduced MIP has 0 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 7 rows, 12 columns, and 24 nonzeros.
Reduced MIP has 0 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solu

In [31]:
# Display results
print(m.get_solve_status())
soln.display()

JobSolveStatus.OPTIMAL_SOLUTION
solution for: 8.1-1
objective: 20200.000
x_1_3 = 2
x_1_4 = 10
x_2_2 = 9
x_2_3 = 8
x_3_1 = 10
x_3_2 = 1


In [ ]:
m.g